In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist 

In [2]:
def kmeans(x, k, no_of_iterations):
    idx = np.random.choice(len(x), k, replace=False)
    print(idx)
    #Randomly choosing Centroids 
    centroids = x[idx, :] #Step 1
     
    #finding the distance between centroids and all the data points
    distances = cdist(x, centroids ,'euclidean') #Step 2
     
    #Centroid with the minimum Distance
    points = np.array([np.argmin(i) for i in distances]) #Step 3
    
    for i in range(k):
        points[idx[i]] = i
        
    
    #Repeating the above steps for a defined number of iterations
    for itr in range(no_of_iterations): 
        centroids = []
        print(f"*** Iteration {itr} ***")
        for idx in range(k):
            #Updating Centroids by taking mean of Cluster it belongs to
            print(x[points==idx])
            temp_cent = x[points==idx].mean(axis=0) 
            centroids.append(temp_cent)
 
        centroids = np.vstack(centroids) #Updated Centroids 
         
        distances = cdist(x, centroids ,'euclidean')
        points = np.array([np.argmin(i) for i in distances])
         
    return centroids

In [3]:
def padImageForPatches(img, patch_shape):
    return cv2.copyMakeBorder(src=img, left=0, right=patch_shape[1]*((img.shape[1]+31)//patch_shape[1])-img.shape[1], top=0, bottom=patch_shape[0]*((img.shape[0]+31)//patch_shape[0])-img.shape[0], borderType=cv2.BORDER_REFLECT)

def divideIntoPatches(img, patch_shape):
    img = padImageForPatches(img, patch_shape)
    img_shape = img.shape
    patch_array = np.empty((int((img_shape[0]*img_shape[1])/(patch_shape[0]*patch_shape[1])), patch_shape[0], patch_shape[1], 3), dtype=np.int32)
    for i in range(0, int(img_shape[0]/patch_shape[0])):
        for j in range(0, int(img_shape[1]/patch_shape[1])):
            patch_array[i*int(img_shape[1]/patch_shape[1]) + j] = img[i*patch_shape[0]:i*patch_shape[0]+patch_shape[0], j*patch_shape[1]:j*patch_shape[1]+patch_shape[1], :]

    return patch_array

def getColorHistFeatures(img, n_bins=8):
    img = img//int(256/n_bins)
    hist = np.bincount(img[:, :, 0].ravel(), minlength=8)
    hist = np.append(hist, np.bincount(img[:, :, 1].ravel(), minlength=8))
    hist = np.append(hist, np.bincount(img[:, :, 2].ravel(), minlength=8))
    return hist


def getHistForPatches(img, patch_shape):
    patch_array = divideIntoPatches(img, patch_shape)
    n, h, w, c = patch_array.shape
    hist_array = np.empty((n, 24), dtype=np.int32)
    for i in range(0, n):
        hist_array[i] = getColorHistFeatures(patch_array[i])
    return hist_array

def loadImagesFromDir(dir_path):
    images = {}
    
    for folder in os.listdir(dir_path):
        img_arr = []
        folder_path = dir_path + "/" + folder
        for file in os.listdir(folder_path):
            img_path = folder_path + "/" + file
            img = cv2.imread(img_path)
            if img is not None:
                img_arr.append(img)
        images[folder] = img_arr
    
    return images

def getHistForAllImages(images, patch_shape):
    images_hist_features = {}

    for key, img_arr in images.items():
        hist24_arr = []
        for img in img_arr:
            hist24_arr.append(getHistForPatches(img, patch_shape))
        images_hist_features[key] = hist24_arr
    
    return images_hist_features

In [4]:
train_images = loadImagesFromDir("Group21/Classification/Image_Group21/train")
test_images = loadImagesFromDir("Group21/Classification/Image_Group21/test")

In [5]:
train_imgs_hist = getHistForAllImages(train_images, patch_shape=(32, 32))
test_imgs_hist = getHistForAllImages(test_images, patch_shape=(32, 32))

In [6]:
train_imgs_hist_flattened = np.concatenate([y for x in train_imgs_hist.values() for y in x])

In [7]:
# testing
print(train_imgs_hist_flattened.shape)

(150752, 24)


In [8]:
# centroids = kmeans(train_imgs_hist_flattened, 32, 100)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=32, random_state=0).fit(train_imgs_hist_flattened)
centroids = kmeans.cluster_centers_

In [9]:
print(kmeans.labels_)
print(np.bincount(kmeans.labels_))
print(kmeans.inertia_)
print(kmeans.n_iter_)

[ 0  0  0 ... 28  6  6]
[15515 11457  1963  5629  5314  3873  6691  4484  6180  1540 12314  5535
  4195  4305  3462  2853  2409  3358  3170  6394  3538  1702  6742   748
  1235  8638  5350  2095  3945  2815  1395  1908]
39698874647.048
76


In [10]:
def BoVW_image_feature_vector(img_col_hist, centroids):
    distances = cdist(img_col_hist, centroids ,'euclidean')
    points = np.array([np.argmin(i) for i in distances])
    feature = np.bincount(points, minlength=len(centroids))/points.shape[0]
    return feature

In [11]:
list(train_imgs_hist.values())[0][0][1]

array([  0,   0,   0,   0,  11,  70,  23, 920,   0,   0,   0,   0,  13,
        72,  20, 919,   0,   0,   0,   3,  49,  43,  36, 893])

In [12]:
print(BoVW_image_feature_vector(list(train_imgs_hist.values())[0][0], centroids))
print(BoVW_image_feature_vector(list(train_imgs_hist.values())[0][0], centroids).sum())

[0.05769231 0.16346154 0.         0.         0.         0.
 0.02884615 0.         0.00961538 0.03846154 0.15384615 0.00961538
 0.04807692 0.         0.         0.05769231 0.         0.04807692
 0.10576923 0.01923077 0.03846154 0.         0.11538462 0.
 0.         0.06730769 0.         0.         0.         0.03846154
 0.         0.        ]
1.0


In [13]:
img_BoVW_train = {}
for img_type, img_hist_array in train_imgs_hist.items():
    img_BoVW = np.empty((len(img_hist_array), 32), dtype=np.float64)
    for i in range(len(img_hist_array)):
        img_BoVW[i] = BoVW_image_feature_vector(img_hist_array[i], centroids)
    img_BoVW_train[img_type] = img_BoVW

In [14]:
img_BoVW_test = {}
for img_type, img_hist_array in test_imgs_hist.items():
    img_BoVW = np.empty((len(img_hist_array), 32), dtype=np.float64)
    for i in range(len(img_hist_array)):
        img_BoVW[i] = BoVW_image_feature_vector(img_hist_array[i], centroids)
    img_BoVW_test[img_type] = img_BoVW

In [15]:
print(img_BoVW_train["batters_box"][0].shape)
print(img_BoVW_train["batters_box"][0].sum())

(32,)
1.0


In [16]:
from MLFFNN_lib import MLFFNN

c1 = img_BoVW_train['batters_box']
c2 = img_BoVW_train['racecourse']
c3 = img_BoVW_train['tree_farm']
t1 = img_BoVW_test['batters_box']
t2 = img_BoVW_test['racecourse']
t3 = img_BoVW_test['tree_farm']


c1 = np.append(c1, np.full((c1.shape[0], 3), [1, 0, 0]), axis=1)
c2 = np.append(c2, np.full((c2.shape[0], 3), [0, 1, 0]), axis=1)
c3 = np.append(c3, np.full((c3.shape[0], 3), [0, 0, 1]), axis=1)
t1 = np.append(t1, np.full((t1.shape[0], 3), [1, 0, 0]), axis=1)
t2 = np.append(t2, np.full((t2.shape[0], 3), [0, 1, 0]), axis=1)
t3 = np.append(t3, np.full((t3.shape[0], 3), [0, 0, 1]), axis=1)
data = np.concatenate((c1, c2, c3), axis=0)
data1 = np.concatenate((t1, t2, t3), axis=0)
np.random.shuffle(data)
np.random.shuffle(data1)

test_data = data
train_data = data1

In [17]:
data[40]

array([0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.025 , 0.    ,
       0.    , 0.    , 0.075 , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.1875, 0.    , 0.2   , 0.375 , 0.    , 0.1375, 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 1.    ])

In [18]:
layer_sizes = [[32, 20, 10, 3], [32, 40, 20, 3], [32, 64, 16, 3], [32, 64, 128, 16, 3]]

In [37]:
for layer_size in layer_sizes:
    net = MLFFNN(layers_size = layer_size)
    print(f"***** network = {layer_size} *****\n\n")
    net.train(train_data[:, :-3], train_data[:, 32:], test_data[:, :-3], test_data[:, 32:], epochs=5000, eta=.8)
    plt.plot(net.MSE, label = f"network = {layer_size}")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Mean Squared error")
plt.plot()

***** network = [32, 20, 10, 3] *****




ValueError: operands could not be broadcast together with shapes (149,3) (149,3,1) 